In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_excel('11.2 WTI.xlsx', 'Spot', index_col='Date')
df2 = pd.read_excel('11.2 WTI.xlsx', 'Front', index_col='Date')
df3 = pd.read_excel('11.2 WTI.xlsx', 'Second', index_col='Date')

In [3]:
df = pd.concat([df1, df2, df3], axis=1)
df.tail()

,Spot,Front,Second
Date,,,
2010-01-08,82.75,82.75,83.30
2010-01-07,82.66,82.66,83.19
2010-01-06,83.18,83.18,83.75
2010-01-05,81.77,81.77,82.41
2010-01-04,81.51,81.51,82.12


In [4]:
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')
df[10:15]

,Spot,Front,Second
Date,,,
2017-12-13,56.60,56.60,56.59
2017-12-12,57.14,57.14,57.16
2017-12-11,57.99,57.99,58.05
2017-12-08,57.36,57.36,57.44
2017-12-07,56.69,56.69,56.75


In [5]:
df_e = pd.read_excel('11.2 WTI.xlsx', 'Expiry')
df_e[120:125]

,Ticker,Month,Last Trade,Last Price
120,CLG10,2017-02-10,01/20/10,77.62
121,CLH10,2017-03-10,02/22/10,80.16
122,CLJ10,2017-04-10,03/22/10,81.25
123,CLK10,2017-05-10,04/20/10,83.45
124,CLM10,2017-06-10,05/20/10,68.01


In [6]:
# 만기일 추출
df_e['Last Trade'].head()

0    01/20/00
1    02/22/00
2    03/21/00
3    04/19/00
4    05/22/00
Name: Last Trade, dtype: object

In [7]:
# 만기일을 날짜 형식으로 변환
pd.to_datetime(df_e['Last Trade']).head()

0   2000-01-20
1   2000-02-22
2   2000-03-21
3   2000-04-19
4   2000-05-22
Name: Last Trade, dtype: datetime64[ns]

In [8]:
# 추출한 만기일 정보를 Series에 저장
expiry = pd.Series(pd.to_datetime(df_e['Last Trade']))
expiry.head()

0   2000-01-20
1   2000-02-22
2   2000-03-21
3   2000-04-19
4   2000-05-22
Name: Last Trade, dtype: datetime64[ns]

In [9]:
pd.to_datetime(expiry.values)

DatetimeIndex(['2000-01-20', '2000-02-22', '2000-03-21', '2000-04-19',
               '2000-05-22', '2000-06-20', '2000-07-20', '2000-08-22',
               '2000-09-20', '2000-10-20',
               ...
               '2023-08-22', '2023-09-20', '2023-10-20', '2023-11-20',
               '2024-05-21', '2024-11-20', '2025-05-20', '2025-11-20',
               '2026-05-19', '2026-11-20'],
              dtype='datetime64[ns]', length=293, freq=None)

In [10]:
print(df.index)

DatetimeIndex(['2017-12-28', '2017-12-27', '2017-12-26', '2017-12-22',
               '2017-12-21', '2017-12-20', '2017-12-19', '2017-12-18',
               '2017-12-15', '2017-12-14',
               ...
               '2010-01-15', '2010-01-14', '2010-01-13', '2010-01-12',
               '2010-01-11', '2010-01-08', '2010-01-07', '2010-01-06',
               '2010-01-05', '2010-01-04'],
              dtype='datetime64[ns]', name='Date', length=2014, freq=None)


In [11]:
df['Expiry'] = np.nan
df.head()

,Spot,Front,Second,Expiry
Date,,,,
2017-12-28,59.84,59.84,59.87,NaN
2017-12-27,59.64,59.64,59.69,NaN
2017-12-26,59.97,59.97,60.00,NaN
2017-12-22,58.42,58.47,58.54,NaN
2017-12-21,58.33,58.36,58.40,NaN


In [12]:
# Trading day 중 만기일 찾아서 표시하기
for d in df.index:
    if d in pd.to_datetime(expiry.values):
        df.loc[d].Expiry = 1
df[10:15] 

,Spot,Front,Second,Expiry
Date,,,,
2017-12-13,56.60,56.60,56.59,NaN
2017-12-12,57.14,57.14,57.16,NaN
2017-12-11,57.99,57.99,58.05,NaN
2017-12-08,57.36,57.36,57.44,NaN
2017-12-07,56.69,56.69,56.75,NaN


In [13]:
# 현물에 투자했다면
# 손익 = 현재가격 - 초기가격
price_now = float(df['2017-12-28']['Spot'])
price_before = float(df['2010-01-04']['Spot'])
profit = (price_now - price_before) * 1000
print('손익 (현재가격 - 초기가격) : ', '{:,.2f}'.format(profit))

손익 (현재가격 - 초기가격) :  -21,670.00


In [14]:
# 선물에 투자했다면
# 현재가격 - 초기가격 - 롤오버 비용
roll_over_cost_acc = 0
for d in df.index:
    if (df.loc[d].Expiry == 1):
        # 롤오버 비용 = 차근월물 가격 - 최근월물 가격
        roll_over_cost = df.loc[d].Second - df.loc[d].Front
        roll_over_cost_acc = roll_over_cost_acc + roll_over_cost
        
print('누적 롤오버 비용 :', '{:,.2f}'.format(roll_over_cost_acc))

누적 롤오버 비용 : 40.68


In [15]:
price_now = float(df['2017-12-28']['Front'])
price_before = float(df['2010-01-04']['Front'])
profit = (price_now - price_before - roll_over_cost_acc) * 1000
print('누적 손익', '{:,.2f}'.format(profit))

누적 손익 -62,350.00
